In [44]:
# Installation de DuckDB dans le notebook
!pip install duckdb

import duckdb



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
def script_is_running_on_colab() -> bool:
    """
    Détecte si le script s'exécute dans Google Colab.
    """
    try:
        import google.colab  # type: ignore
        return True
    except ImportError:
        return False
if script_is_running_on_colab():

    PATH_TO_PROJECT = "./Atelier_entrepot_de_donnee_epsi/"
    !rm -rf Atelier_entrepot_de_donnee_epsi
    !git clone https://github.com/lhuissier4/Atelier_entrepot_de_donnee_epsi.git
    # Test de connexion
    conn = duckdb.connect(database=':memory:')
    print("DuckDB est prêt et fonctionne sans backend !")
else:
    PATH_TO_PROJECT="./"
    conn = duckdb.connect(database='my_database.duckdb')
    print("DuckDB est prêt et fonctionne avec backend !")

print(PATH_TO_PROJECT)

DuckDB est prêt et fonctionne avec backend !
./


In [46]:
import pandas as pd
df_lobby_stats_player = pd.read_csv(
    PATH_TO_PROJECT+"dataset/tb_lobby_stats_player.csv",
    sep=",",
    encoding="utf-8"
)
df_medalha = pd.read_csv(
    PATH_TO_PROJECT+"dataset/tb_medalha.csv",
    sep=",",
    encoding="utf-8"
)
df_players = pd.read_csv(
    PATH_TO_PROJECT+"dataset/tb_players.csv",
    sep=",",
    encoding="utf-8"
)
df_players_medalhar = pd.read_csv(
    PATH_TO_PROJECT+"dataset/tb_players_medalha.csv",
    sep=",",
    encoding="utf-8"
)

In [47]:
from pathlib import Path
def execute_sql_folder(connection, folder_path: str) -> None:
    """
    Exécute tous les fichiers .sql d'un dossier DuckDB dans l'ordre alphabétique.
    """

    folder = Path(folder_path)

    # Vérification du dossier
    if not folder.exists() or not folder.is_dir():
        raise NotADirectoryError(f"Invalid SQL folder: {folder_path}")

    # Récupération et tri des fichiers .sql
    sql_files = sorted(folder.glob("*.sql"))

    if not sql_files:
        raise FileNotFoundError(f"No SQL files found in folder: {folder_path}")

    # Exécution des fichiers
    for sql_file in sql_files:
        sql_code = sql_file.read_text(encoding="utf-8")

        connection.execute(sql_code)
        print(f"Executed: {sql_file.name}")


In [48]:
execute_sql_folder(conn,PATH_TO_PROJECT+"migrations/down")

Executed: 01_down_players_medals.sql
Executed: 02_down_lobby_stats_player.sql
Executed: 04_down_medals.sql
Executed: 05_down_players.sql


In [49]:
execute_sql_folder(conn,PATH_TO_PROJECT+"migrations/up")

Executed: 01_create_table_players.sql
Executed: 02_create_table_medals.sql
Executed: 03_create_table_country.sql
Executed: 04_create_table_map_name.sql
Executed: 05_create_table_madalha_type.sql
Executed: 06_create_table_tb_lobby_stats_player.sql
Executed: 07_create_table_players_medalha.sql


In [50]:
from typing import Dict, Iterable

def load_dataset_to_duckdb(
    connection,
    dataset: pd.DataFrame,
    table_name: str,
    column_mapping: Dict[str, str] | None = None,
    exclude_columns: Iterable[str] | None = None
) -> None:
    """
    Charge un dataset (DataFrame pandas) dans DuckDB,
    avec renommage et exclusion de colonnes.
    """

    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("dataset must be a pandas DataFrame")

    df = dataset.copy()

    # Exclusion des colonnes
    if exclude_columns:
        df = df.drop(columns=list(exclude_columns), errors="ignore")

    # Renommage des colonnes
    if column_mapping:
        df = df.rename(columns=column_mapping)

    # Enregistrement temporaire dans DuckDB
    connection.register("tmp_df", df)

    # Insertion dans la table cible
    connection.execute(f"""
        INSERT INTO {table_name}
        SELECT * FROM tmp_df
    """)

    # Nettoyage
    connection.unregister("tmp_df")

    print(f"Dataset loaded into table {table_name}")


Nettoyage des données

In [51]:
def get_columns_with_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Analyse un DataFrame et retourne les colonnes contenant des valeurs manquantes.

    :param df: DataFrame à analyser
    :return: DataFrame contenant les colonnes avec valeurs manquantes,
             le nombre de valeurs nulles et leur pourcentage
    """
    # Calcul du nombre de valeurs nulles par colonne
    missing_count = df.isnull().sum()

    # Filtrer uniquement les colonnes avec au moins une valeur manquante
    missing_count = missing_count[missing_count > 0]

    # Calcul du pourcentage
    missing_percentage = (missing_count / len(df)) * 100

    # Construction du DataFrame résultat
    result = pd.DataFrame({
        "missing_count": missing_count,
        "missing_percentage": missing_percentage.round(2)
    }).sort_values(by="missing_count", ascending=False)

    return result


In [52]:
# Affichages des valeurs manquantes
print('df_lobby_stats_player',get_columns_with_missing_values(df_lobby_stats_player))
print('df_medalha',get_columns_with_missing_values(df_medalha))
print('df_players',get_columns_with_missing_values(df_players))
print('df_players_medalhar',get_columns_with_missing_values(df_players_medalhar))


df_lobby_stats_player                missing_count  missing_percentage
qtSurvived               705                0.38
qtTrade                  705                0.38
qtHitStomach             705                0.38
qtHitChest               705                0.38
qtHitHeadshot            705                0.38
qtFlashAssist            705                0.38
qtHitLeftAtm             705                0.38
qtHitRightArm            705                0.38
qtHitLeftLeg             705                0.38
qtHitRightLeg            705                0.38
qtHits                   120                0.07
qtLastAlive              120                0.07
qtTk                     120                0.07
qtTkAssist               120                0.07
df_medalha Empty DataFrame
Columns: [missing_count, missing_percentage]
Index: []
df_players          missing_count  missing_percentage
dtBirth            708               26.07
df_players_medalhar               missing_count  missing_percent

In [53]:
print(df_players_medalhar.head())
print(df_players.head())
print(df_players.dtypes)


   id  idPlayer  idMedal dtCreatedAt dtExpiration    dtRemove  flActive
0   1      1982        1  2018-10-04   2016-07-23  2016-07-24         0
1   2      1982        1  2018-12-12   2017-01-11  2017-01-12         0
2   3      1982        1  2019-01-08   2017-03-16  2017-03-17         0
3   4      1982        1  2018-12-09   2017-09-10  2017-09-11         0
4   5      1982        1  2018-10-04   2018-03-19  2018-03-20         0
   idPlayer  flFacebook  flTwitter  flTwitch descCountry dtBirth  \
0         1           0          0         0          cl     NaN   
1         2           0          0         0          br     NaN   
2         3           0          0         0          br     NaN   
3         4           0          0         0          br     NaN   
4         5           0          0         0          ar     NaN   

        dtRegistration  
0  2021-03-19 21:31:39  
1  2020-05-06 19:28:29  
2  2020-01-28 13:00:38  
3  2017-05-31 16:13:51  
4  2021-02-08 11:57:21  
idPlayer 

In [54]:


import pandas as pd
import numpy as np

def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remplace :
    - les valeurs nulles des colonnes numériques par 0
    - les valeurs nulles des colonnes datetime par 1970-01-01

    :param df: DataFrame à traiter
    :return: DataFrame modifié
    """

    # Copie pour éviter de modifier l'original
    df = df.copy()

    # Colonnes numériques
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].fillna(0)


    return df

df_players = fill_missing_values(df_players)
for col in ["dtRegistration","dtBirth"]:
  df_players[col] = df_players[col].fillna("1970-01-01")


df_lobby_stats_player = fill_missing_values(df_lobby_stats_player)
df_medalha = fill_missing_values(df_medalha)
df_players_medalhar = fill_missing_values(df_players_medalhar)
for col in ["dtExpiration","dtRemove", "dtCreatedAt"]:
    df_players_medalhar[col] = df_players_medalhar[col].fillna("1970-01-01")

print('df_lobby_stats_player',get_columns_with_missing_values(df_lobby_stats_player))
print('df_medalha',get_columns_with_missing_values(df_medalha))
print('df_players',get_columns_with_missing_values(df_players))
print('df_players_medalhar',get_columns_with_missing_values(df_players_medalhar))

df_lobby_stats_player Empty DataFrame
Columns: [missing_count, missing_percentage]
Index: []
df_medalha Empty DataFrame
Columns: [missing_count, missing_percentage]
Index: []
df_players Empty DataFrame
Columns: [missing_count, missing_percentage]
Index: []
df_players_medalhar Empty DataFrame
Columns: [missing_count, missing_percentage]
Index: []


In [55]:
import ast


def normalize_list_column(df: pd.DataFrame, column_name: str):
    """
    Normalise une colonne contenant des listes de valeurs.

    Retourne :
    - df_original_modifie : avec IDs à la place des noms
    - df_dimension : table id / name sans doublon
    """

    df = df.copy()

    # 1️⃣ Exploser la colonne liste
    exploded = df.explode(column_name)

    # 2️⃣ Extraire les valeurs uniques
    unique_values = exploded[column_name].dropna().unique()

    # 3️⃣ Créer la table dimension
    df_dimension = pd.DataFrame({
        "id": range(1, len(unique_values) + 1),
        column_name: unique_values
    })

    # 4️⃣ Créer mapping name -> id
    mapping = dict(zip(df_dimension[column_name], df_dimension["id"]))
    # 5️⃣ Remplacer dans le dataframe original
    df[column_name] = df[column_name].map(mapping)

    return df, df_dimension
df_lobby_stats_player, df_descMapName = normalize_list_column(df_lobby_stats_player,"descMapName")
df_players, df_country = normalize_list_column(df_players, "descCountry")
df_medalha, df_medalha_type = normalize_list_column(df_medalha, "descMedal")

In [56]:
print(df_lobby_stats_player.head())

   idLobbyGame  idPlayer  idRoom  qtKill  qtAssist  qtDeath  qtHs  \
0            1         1       1       5         1       16     2   
1            2         1       2      24         3       18     6   
2            3         2       3       6         4       23     2   
3            3       391   27508      10         5       20     4   
4            4         2       4       8         4       26     6   

   qtBombeDefuse  qtBombePlant  qtTk  ...  qtFlashAssist  qtHitHeadshot  \
0              0             0   0.0  ...            0.0            3.0   
1              0             4   0.0  ...            0.0            7.0   
2              0             1   0.0  ...            0.0            3.0   
3              1             0   0.0  ...            0.0            6.0   
4              0             2   0.0  ...            2.0            8.0   

   qtHitChest  qtHitStomach  qtHitLeftAtm  qtHitRightArm  qtHitLeftLeg  \
0        13.0           4.0           2.0            2.0    

In [57]:
load_dataset_to_duckdb(conn,df_country,"COUNTRY",{"descCountry":"country"})
load_dataset_to_duckdb(conn,df_descMapName,"MAP_NAME",{"descMapName":"mapName"})
load_dataset_to_duckdb(conn,df_medalha_type,"MEDALHA_TYPE",{"descMedal":"type"})

#load_dataset_to_duckdb(conn,df_players,"PLAYERS",{"idPlayer":"id","dtBirth":"dateBirth","dtRegistration":"dateRegistration","descCountry":"idCountry"},{"flFacebook","flTwitter","flTwitch"})
#load_dataset_to_duckdb(conn,df_medalha,"MEDALHA",{"descMedal":"medalName","descTypeMedal":"idMedalType"})
#load_dataset_to_duckdb(conn,df_lobby_stats_player,"LOBBY_STATS_PLAYER",{"descMapName":"idDescMapName"})
#load_dataset_to_duckdb(conn,df_players_medalhar,"PLAYERS_MEDALHA")
conn.close()

ConstraintException: Constraint Error: Duplicate key "id: 1" violates primary key constraint.

##### FIN DE LA CREATION DE LA BASE DE DONNEES #####

# Atelier 2 :
I - Récupération des données dans la base

In [136]:
!pip install pyarrow
conn = duckdb.connect(database='my_database.duckdb')


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


import des données

In [137]:
df_lobby_stats_player_bdd = conn.execute("SELECT * FROM LOBBY_STATS_PLAYER").fetch_arrow_table().to_pandas()
df_medalha_bdd = conn.execute("SELECT * FROM MEDALHA").fetch_arrow_table().to_pandas()
df_players_bdd = conn.execute("SELECT * FROM PLAYERS").fetch_arrow_table().to_pandas()
df_players_medalhar_bdd = conn.execute("SELECT * FROM PLAYERS_MEDALHA").fetch_arrow_table().to_pandas()


affichage des dataframes

In [138]:
print(f"df_lobby_stats_player_bdd\n{df_lobby_stats_player_bdd.head()}")
print(f"df_medalha_bdd\n{df_medalha_bdd.head()}")
print(f"df_players_bdd\n{df_players_bdd.head()}")
print(f"df_players_medalhar_bdd\n{df_players_medalhar_bdd.head()}")

df_lobby_stats_player_bdd
   idLobbyGame  idPlayer  idRoom  qtKill  qtAssist  qtDeath  qtHs  \
0            1         1       1       5         1       16     2   
1            2         1       2      24         3       18     6   
2            3         2       3       6         4       23     2   
3            3       391   27508      10         5       20     4   
4            4         2       4       8         4       26     6   

   qtBombeDefuse  qtBombePlant  qtTk  ...  qtFlashAssist  qtHitHeadshot  \
0              0             0     0  ...              0              3   
1              0             4     0  ...              0              7   
2              0             1     0  ...              0              3   
3              1             0     0  ...              0              6   
4              0             2     0  ...              2              8   

   qtHitChest  qtHitStomach  qtHitLeftArm  qtHitRightArm  qtHitLeftLeg  \
0          13             4       

In [61]:
conn.close()